In [ ]:
#advance CNN
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303427
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.297500
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.296784
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.293174
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.298241
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.299460
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.289486
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.288715
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.294914
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.285673
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.259745
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.235617
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.210759
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.131562
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.082076
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.954512
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.727958
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.385813
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.192817
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.913096
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.144969
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.185812
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.082270
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.124936
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.157429
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.109602
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.109141
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.187056
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.093484
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.084477
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.088204
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.089769
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.138585
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.215796
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.141584
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.161849
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.095105
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.074730
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.132037
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.102515


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.045041
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.095266
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.066534
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.153523
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.009167
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.088760
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.078722
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.070579
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.040649
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.024543
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.100452
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.068172
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.060882
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.086839
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.058274
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.015147
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.035134
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.082072
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.045588
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.046763


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.120528
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.008875
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.028829
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.023318
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.018636
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.014117
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.009999
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.028450
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.006345
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.057188
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.032176
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.051854
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.097394
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.126580
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.063101
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.051553
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.045342
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.156445
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.036046
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.087380


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.010109
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.030910
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.034403
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.054133

Test set: Average loss: 0.0534, Accuracy: 9838/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.005052
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.071378
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.053379
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.026034
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.016039
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.020998
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.084050
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.073344
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.028799
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.317038
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.018030
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.294376
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.034332
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.037583
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.058619
Train Epoch: 8 [9